In [94]:
import pandas as pd


In [111]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df=pd.DataFrame(df[0])
df.head()
df.drop(df.loc[df['Borough']=='Not assigned'].index,inplace=True)

In [112]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [119]:
def concat_neighbourhoods(dfp):
    D = dict.fromkeys(df['Postcode'].unique())
    for item in dfp:
        hds = ", ".join(list(item[1]['Neighbourhood'].unique()))
        bor = ", ".join(list(item[1]['Borough'].unique()))
        D[item[0]] = (bor, hds)
        
    new_df = pd.DataFrame.from_dict(D, orient='index') 
    new_df.reset_index(inplace=True) 
    new_df.columns = ['Postcode', 'Borough', 'Neighbourhood'] 
    return new_df


df1 = df.groupby(['Postcode'])  
df = concat_neighbourhoods(df1)  
df.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [125]:
df.shape

(103, 3)

In [126]:
geospatial='https://cocl.us/Geospatial_data'
geoxy=pd.read_csv(geospatial)
geoxy.columns=['Postcode','Latitude','Longitude']

In [128]:
df_total=df.merge(geoxy,on='Postcode')

In [129]:
df_total.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


In [140]:
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors


In [137]:
kclusters=5
cluster_data=df_total[['Latitude','Longitude']]
kmeans=KMeans(n_clusters=k,random_state=0).fit(cluster_data)
labels=kmeans.labels_

In [138]:
df_total['labels']=labels
df_total.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,labels
0,M3A,North York,Parkwoods,43.753259,-79.329656,4
1,M4A,North York,Victoria Village,43.725882,-79.315572,4
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,2
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,3
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,2


In [142]:
latitude = 43.6532
longitude = -79.3832
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_total['Latitude'], df_total['Longitude'], df_total['Neighbourhood'], df_total['labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters